## This is an example of batch processing with subfolder hierarchy
Each subfolder in the path is a variable, that will be added as a separate column in the output table.
In the input the choosen option for analysis can be specified. 

## Add package to python path and import function (s)

In [1]:
import sys
import os

# Manually set the parent directory path
notebook_dir = os.getcwd()  # This gets the current working directory of the notebook
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))

# Add the parent directory to the system path
sys.path.insert(0, parent_dir)

#import an example function from package 
from flim_processing import analyze_single_fitted_channel,generate_small_report_fig,analyze_single_fastflim_channel,in_folder_loop

## Provide an absolut path to the highest hierarchy folder

In [3]:
from pathlib import Path
global_path=Path(r"G:\SP8_FLIM\0_Workflow_OPTIM\5HT")

In [7]:
# specifies exact function that will be used in analysis 
selected_analysis_function = analyze_single_fitted_channel

In [4]:
import pandas as pd
from pathlib import Path
import natsort
from skimage.io import imsave

# Create the folder where all the outputs will be collected 
output_folder_path = global_path / 'Outputs'
output_folder_path.mkdir(exist_ok=True)

# Initialize master DataFrames and other data structures outside the loop
master_results = pd.DataFrame([])
master_reports = pd.DataFrame([])

# Iterate over all folders in the global_path directory
for day_path in global_path.iterdir():
    if day_path.is_dir() and day_path.stem != 'Outputs':
        
        date = day_path.stem
        print(f"Processing date folder: {date}")
        
        result_per_date = pd.DataFrame([])

        # treatment level
        for treatment_path in day_path.iterdir():
            if treatment_path.is_dir(): 
                print(treatment_path)
                treatment = treatment_path.stem
                print(treatment_path.stem)
                print('\n')

                for clone_path in treatment_path.iterdir():
                    if clone_path.is_dir():  # Check if clone_path is a directory
                        
                        outputimage_folder_path = clone_path / 'Output_images'
                        outputimage_folder_path.mkdir(exist_ok=True)
                        
                        print(clone_path)
                        clone = clone_path.stem
                        print(clone_path.stem)
                        print('\n')

                        # local DataFrame to store data for the current folder
                        result_per_folder = pd.DataFrame([])
                        report_per_folder = pd.DataFrame([])

                        # Here we build a paired_path nested list
                        paired_path_list = []
                        paired_paths = []

                        # Create paired paths
                        for file_path in natsort.natsorted(clone_path.iterdir()):
                            if file_path.suffix == '.tif': 
                                paired_paths.append(file_path) 
                                if len(paired_paths) == 2:
                                    paired_path_list.append(paired_paths)
                                    paired_paths = []

                                    
                        # Process each pair of files in the current folder
                        for paired_path in paired_path_list:
                            result_per_image, report_per_image, multichannel_image = selected_analysis_function(paired_path[1], paired_path[0])

                            # Add identification to the result table (image level)
                            result_per_image['File Name'] = paired_path[1].stem
                            report_per_image['File Name'] = paired_path[1].stem


                            #start creating report images 
                            # Construct the desired identification string for images 
                            identification = f"{date}_{paired_path[1].parent.name}_{paired_path[1].stem}"

                            #construct report images and save them 
                            #small_report_fig = generate_small_report_fig(multichannel_image,identification,output_folder_path)

                            # Concatenate the filtered results to the local table
                            result_per_folder = pd.concat([result_per_folder, result_per_image])
                            report_per_folder = pd.concat([report_per_folder, report_per_image]) 
                            
                            #output images. only 1 chanel from param image (last chanel) 
                            image_name='param_map_' +paired_path[1].stem+'.tif'
                            imsave(outputimage_folder_path / image_name, multichannel_image[...,-1])
              

                        result_per_folder['Treatment'] = treatment
                        result_per_folder['Clone'] = clone
                        result_per_folder['Date'] = date
                        result_per_folder['selected_analysis'] =selected_analysis_function.__name__

                        report_per_folder['Treatment'] = treatment
                        report_per_folder['Clone'] = clone
                        report_per_folder['Date'] = date
                        report_per_folder['selected_analysis'] =selected_analysis_function.__name__
                        
                        #optional troubleshooting savinf per specific treatment 
                        #report_per_folder.to_csv(output_folder_path / f"{date}_{clone}_{treatment}.csv")
                        # Concatenate the local table to the master DataFrame
                        master_results = pd.concat([master_results, result_per_folder])
                        master_reports = pd.concat([master_reports, report_per_folder])


print("Saving master table...")
# Save the master DataFrame as a CSV file
master_results.to_csv(output_folder_path / "master_table.csv")

print("Saving table of reports...")
# Save the full table of stat reports per each image in the tree folders
master_reports.to_csv(output_folder_path / "master_reports.csv")

print("Processing complete!")


Processing date folder: 2023-02-13
G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-13\5HT_0
5HT_0


G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-13\5HT_0\cl1_pcDNA_NT
cl1_pcDNA_NT




C:\Users\med\AppData\Local\Temp\ipykernel_6252\562123178.py:81: UserWarning: G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-13\5HT_0\cl1_pcDNA_NT\Output_images\param_map_2.tif is a low contrast image
  imsave(outputimage_folder_path / image_name, multichannel_image[...,-1])
C:\Users\med\AppData\Local\Temp\ipykernel_6252\562123178.py:81: UserWarning: G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-13\5HT_0\cl1_pcDNA_NT\Output_images\param_map_4.tif is a low contrast image
  imsave(outputimage_folder_path / image_name, multichannel_image[...,-1])
C:\Users\med\AppData\Local\Temp\ipykernel_6252\562123178.py:81: UserWarning: G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-13\5HT_0\cl1_pcDNA_NT\Output_images\param_map_5.tif is a low contrast image
  imsave(outputimage_folder_path / image_name, multichannel_image[...,-1])


G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-13\5HT_0\cl1_VMAT_NT
cl1_VMAT_NT




C:\Users\med\AppData\Local\Temp\ipykernel_6252\562123178.py:81: UserWarning: G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-13\5HT_0\cl1_VMAT_NT\Output_images\param_map_vmat-0mm-1.tif is a low contrast image
  imsave(outputimage_folder_path / image_name, multichannel_image[...,-1])
C:\Users\med\AppData\Local\Temp\ipykernel_6252\562123178.py:81: UserWarning: G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-13\5HT_0\cl1_VMAT_NT\Output_images\param_map_vmat-0mm-4.tif is a low contrast image
  imsave(outputimage_folder_path / image_name, multichannel_image[...,-1])


G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-13\5HT_500
5HT_500


G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-13\5HT_500\cl1_VMAT_500
cl1_VMAT_500




...

C:\Users\med\AppData\Local\Temp\ipykernel_6252\562123178.py:81: UserWarning: G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-07\5HT_500\cl1_VMAT_500\Output_images\param_map_cl1-5ht-1.tif is a low contrast image
  imsave(outputimage_folder_path / image_name, multichannel_image[...,-1])
C:\Users\med\AppData\Local\Temp\ipykernel_6252\562123178.py:81: UserWarning: G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-07\5HT_500\cl1_VMAT_500\Output_images\param_map_cl1-5ht-4.tif is a low contrast image
  imsave(outputimage_folder_path / image_name, multichannel_image[...,-1])
C:\Users\med\AppData\Local\Temp\ipykernel_6252\562123178.py:81: UserWarning: G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-07\5HT_500\cl1_VMAT_500\Output_images\param_map_cl1-5ht-5.tif is a low contrast image
  imsave(outputimage_folder_path / image_name, multichannel_image[...,-1])


G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-07\5HT_500\cl2_VMAT_500
cl2_VMAT_500




C:\Users\med\AppData\Local\Temp\ipykernel_6252\562123178.py:81: UserWarning: G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-07\5HT_500\cl2_VMAT_500\Output_images\param_map_cl2-5ht-1.tif is a low contrast image
  imsave(outputimage_folder_path / image_name, multichannel_image[...,-1])
C:\Users\med\AppData\Local\Temp\ipykernel_6252\562123178.py:81: UserWarning: G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-07\5HT_500\cl2_VMAT_500\Output_images\param_map_cl2-5ht-2.tif is a low contrast image
  imsave(outputimage_folder_path / image_name, multichannel_image[...,-1])
C:\Users\med\AppData\Local\Temp\ipykernel_6252\562123178.py:81: UserWarning: G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-07\5HT_500\cl2_VMAT_500\Output_images\param_map_cl2-5ht-3.tif is a low contrast image
  imsave(outputimage_folder_path / image_name, multichannel_image[...,-1])


Saving master table...
Saving table of reports...
Processing complete!


C:\Users\med\AppData\Local\Temp\ipykernel_6252\562123178.py:81: UserWarning: G:\SP8_FLIM\0_Workflow_OPTIM\5HT\2023-02-07\5HT_500\cl2_VMAT_500\Output_images\param_map_cl2-5ht-4.tif is a low contrast image
  imsave(outputimage_folder_path / image_name, multichannel_image[...,-1])
